In [7]:
import pandas as pd
import datetime
import re

# 약간의 전처리 과정

In [8]:
df = pd.read_csv('first_model_data.csv')
temp = pd.read_csv('youtube.csv')

In [9]:
# 시작과 끝을 분리해준다.
week = temp['주간'].str.split('-')
# 원래 데이터와 형태 맞춰주기 위해
# 그리고 유튜브 주기랑 가온차트 주기가 다르다. 
# 5를 빼주면 가온차트와 같아진다. 
start = []
end = []
for i in week:
    start.append(str(datetime.datetime.strptime(i[0],'%Y%m%d') - datetime.timedelta(days = 5))[0:10])
    end.append(str(datetime.datetime.strptime(i[1],'%Y%m%d') - datetime.timedelta(days = 5))[0:10])

In [10]:
# 전처리한 값 추가
temp['start'] = start
temp['end'] = end

In [11]:
# song과 날짜로 합칠거라서 song 전처리 해준다 
# 이유: 이름이 다른 사람들이 존재 ex) 조정석 == Jo Jung-suk
song = []
for i in temp['song']:
    a = re.split('[(-)]+' ,i)[0]
    song.append(a.strip())

In [12]:
temp['song'] = song
temp.head()

,주간,순위,song,name,lastweek_rank,지속기간,변경,조회수,start,end
0,20200417-20200423,1위,WANNABE,ITZY,지난주 #1,7 weeks,-9.5%,2.48M,2020-04-12,2020-04-18
1,20200417-20200423,2위,덤더럼,에이핑크,지난주 #4,2 weeks,30.1%,2.1M,2020-04-12,2020-04-18
2,20200417-20200423,3위,돌덩이,하현우,지난주 #2,9 weeks,-16.4%,1.66M,2020-04-12,2020-04-18
3,20200417-20200423,4위,시작,가호,지난주 #5,11 weeks,-10.6%,1.38M,2020-04-12,2020-04-18
4,20200417-20200423,5위,아로하,Jo Jung-suk,지난주 #11,4 weeks,26.7%,1.32M,2020-04-12,2020-04-18


In [13]:
# 필요없는 값 제거
df_for_merge = temp.drop(columns = ['주간','lastweek_rank','지속기간','변경','name'])

In [14]:
# 이름 변경
df_you = df_for_merge.rename(columns = {'start':'st_day', 'end':'ed_day','song':'name'})

In [15]:
df_you['st_day']

0        2020-04-12
1        2020-04-12
2        2020-04-12
3        2020-04-12
4        2020-04-12
            ...    
10395    2018-04-22
10396    2018-04-22
10397    2018-04-22
10398    2018-04-22
10399    2018-04-22
Name: st_day, Length: 10400, dtype: object

In [16]:
a = []
for i in df_you['st_day']:
    a.append(i.split('-'))

In [17]:
year = []
day = []
month = []
for i in range(0,len(a)):
    years = int(a[i][0][2:4])
    months = int(a[i][1])
    days = int(a[i][2])
    year.append(years)
    month.append(months)
    day.append(days)

In [18]:
df_you['year'] = year
df_you['month'] = month
df_you['day'] = day

In [19]:
df_you = df_you.drop(columns = ['st_day','ed_day'])

In [20]:
순위 = []
조회수 = []
# 조회수 전처리
for i in df_you['조회수']:
    if i == 0:
        조회수.append(0)
    elif i != 0:
        if 'M' in i:
            b = i.replace('M','00000')
            if '.' in b:
                b = b.replace('.','')
            조회수.append(int(b))
        elif 'K' in i:
            b = i.replace('K','0000')
            if '.' in b:
                b = b.replace('.','')
            조회수.append(int(b))

# 순위 전처리
for i in df_you['순위']:
    if i == 0:
        순위.append(i)
    elif i != 0:
        i = i.replace('위','')
        순위.append(int(i))

In [21]:
df_you['순위'] = 순위
df_you['조회수'] = 조회수

In [22]:
df_you

,순위,name,조회수,year,month,day
0,1,WANNABE,24800000,20,4,12
1,2,덤더럼,2100000,20,4,12
2,3,돌덩이,16600000,20,4,12
3,4,시작,13800000,20,4,12
4,5,아로하,13200000,20,4,12
...,...,...,...,...,...,...
10395,96,야생화,1660000,18,4,22
10396,97,북,1650000,18,4,22
10397,98,DINOSAUR,1630000,18,4,22
10398,99,"Not Spring, Love, or Cherry Blossoms",1620000,18,4,22


In [23]:
# 유튜브와 합치기
a = pd.merge(df,df_you,on=['name','year','month','day'],how = 'left')

In [24]:
a

,Unnamed: 0,artist,name,previous_song_rank,st_day,X1,id,production,distributor,ed_day,...,dc_m_recommend,dc_m_views,gg_score,nv_score,ndc_t_numb,ndc_t_rec,ndc_t_view,season,순위,조회수
0,1,(여자)아이들,LATATA,NaN,2018.5.6,235,235,큐브엔터테인먼트,카카오 M,2018.5.12,...,0.0,0.0,59.0,64.3,NaN,NaN,NaN,spring,4.0,19600000.0
1,2,(여자)아이들,LATATA,NaN,2018.5.13,319,319,큐브엔터테인먼트,카카오 M,2018.5.19,...,0.0,0.0,59.0,34.9,NaN,NaN,NaN,spring,NaN,NaN
2,3,(여자)아이들,LATATA,NaN,2018.5.20,416,416,큐브엔터테인먼트,카카오 M,2018.5.26,...,0.0,0.0,23.0,38.1,NaN,NaN,NaN,spring,NaN,NaN
3,4,(여자)아이들,LATATA,NaN,2018.5.27,512,512,큐브엔터테인먼트,카카오 M,2018.6.2,...,0.0,0.0,40.0,23.2,NaN,NaN,NaN,spring,NaN,NaN
4,5,(여자)아이들,LATATA,NaN,2018.6.3,614,614,큐브엔터테인먼트,카카오 M,2018.6.9,...,0.0,0.0,60.0,30.3,NaN,NaN,NaN,summer,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17478,17396,히키,사랑에 연습이 있었다면,NaN,2019.6.23,8701,8701,"꿈의엔진, 디원미디어",카카오 M,2019.6.29,...,0.0,0.0,84.0,36.6,NaN,NaN,NaN,summer,NaN,NaN
17479,17397,히키,사랑에 연습이 있었다면,NaN,2019.6.30,8904,8904,"꿈의엔진, 디원미디어",카카오 M,2019.7.6,...,0.0,0.0,32.0,32.7,NaN,NaN,NaN,summer,NaN,NaN
17480,17398,히키,사랑에 연습이 있었다면,NaN,2019.7.7,9141,9141,"꿈의엔진, 디원미디어",카카오 M,2019.7.13,...,0.0,0.0,31.0,25.1,NaN,NaN,NaN,summer,NaN,NaN
17481,17399,히키,사랑에 연습이 있었다면,NaN,2019.7.14,9364,9364,"꿈의엔진, 디원미디어",카카오 M,2019.7.20,...,0.0,0.0,54.0,21.1,NaN,NaN,NaN,summer,NaN,NaN


In [25]:
df = a.drop_duplicates(['artist','name','st_day','ed_day'],keep='last')

In [26]:
a = []
for i in list(df['장르']):
    b = i.split(' / ')
    a.append(b)
    
genre = []
for i in range(0,len(list(a))):
    genre.append(a[i][1])

df['genre'] = genre

/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [27]:
df = df.drop(columns = ['X1','id','Unnamed: 0','name_artist','song_id','st_day','production','distributor','ed_day','장르','런타임','성별','score','rank','활동유형','wks','week','active_g','top_freq_g'])

In [28]:
df = df.drop(columns = ['dc_t_number','dc_t_recommend','dc_t_views','dc_m_recommend','runtime_g','dc_m_views','active'])

In [29]:
pd.set_option('display.max_columns', 999)

In [30]:
df.loc[df['previous_song_rank'].isnull(),'previous_song_rank'] = 200

In [31]:
df.loc[df['순위'].isnull(),'순위'] = 100
df.loc[df['조회수'].isnull(),'조회수'] = 0

In [32]:
df.loc[df['ndc_t_numb'].isnull(),'ndc_t_numb'] = 0
df.loc[df['ndc_t_rec'].isnull(),'ndc_t_rec'] = 0
df.loc[df['ndc_t_view'].isnull(),'ndc_t_view'] = 0

In [33]:
df.to_csv('first_model_glm.csv')

In [193]:
df = pd.read_csv('first_model_glm.csv',encoding = 'utf8')

In [194]:
df = df.drop(columns = ['artist','name','year' ,'month' ,'day'])

In [195]:
del df['Unnamed: 0']

In [196]:
del df['X1']

In [197]:
del df['you_rank']

In [198]:
del df['previous_song_rank']

In [199]:
df = df.rename(columns = {'조회수':'total_view'})

In [200]:
df = pd.get_dummies(df, columns = ['sex'])
df = pd.get_dummies(df, columns = ['song_type'])
df = pd.get_dummies(df, columns = ['you_rank_g'])

In [201]:
df = pd.get_dummies(df, columns = ['season'])

In [202]:
df = pd.get_dummies(df, columns = ['genre'])

In [203]:
df = pd.get_dummies(df, columns = ['active_type'])

In [204]:
df = pd.get_dummies(df, columns = ['previous_song_rank_g'])

In [205]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns = ['rank_g']),df['rank_g'], test_size = 0.3)

# 사용 예정 모델
- XGboost
- LightGBM
- RandomForest
- Catboost
- Logistic regression

In [206]:
from sklearn.linear_model import LogisticRegression

In [584]:
model = LogisticRegression(random_state=1)

In [585]:
model.fit(X_train,y_train)

/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [586]:
pred_y = model.predict(X_test)

In [94]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [603]:
print("F1 : %.3f" % f1_score(y_test, pred_y, average = 'micro'))

F1 : 0.474


In [134]:
from sklearn.ensemble import RandomForestClassifier

In [209]:
model = RandomForestClassifier(random_state =1, n_estimators = 500)

In [210]:
model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [211]:
pred_y = model.predict(X_test)

In [212]:
print("F1 : %.3f" % f1_score(y_test, pred_y, average = 'micro'))

F1 : 0.602


In [49]:
from xgboost import XGBClassifier

/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [149]:
# XGB는 생각보다 성능이 낮다... 아마 파라미터 조정을 해야될듯...
model = XGBClassifier(n_estimators = 500)

In [79]:
from lightgbm import LGBMClassifier

In [207]:
model = LGBMClassifier(n_estimators = 500)

In [208]:
# non-ascii 문제가 생긴다... 해결 방법 찾아봐야할 듯. 
model.fit(X_train,y_train)

LightGBMError: Do not support non-ASCII characters in feature name.

In [151]:
pred_y = model.predict(X_test)
print("F1 : %.3f" % f1_score(y_test, pred_y, average = 'micro'))

F1 : 0.577
